In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import os
import json
import torch
import numpy as np
import time
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import clear_output
from nerfstudio.utils.eval_utils import eval_setup
from plane_nerf.inerf_trainer import load_data_into_trainer
from plane_nerf.inerf_utils import load_eval_image_into_pipeline, inerf


In [8]:
os.chdir('/workspace')
MODEL_PATH = "/workspace/outputs/jackal_color_wheel_training_data_1/plane-nerf/2024-02-19_174621"
DATA_PATH = "/stored_data/jackal_color_wheel_evaluation_data"
GROUND_TRUTH_PATH = os.path.join(DATA_PATH, "ground_truth.json")
SAVE_FILE_NAME = "eval_results_"+str(time.strftime("%Y-%m-%d_%H%M%S"))+".csv"

config_path = os.path.join(MODEL_PATH, "config.yml")
config, pipeline, _, _ = eval_setup(
                        Path(config_path),
                        test_mode="inference",
                    )
transform_file_path = "transforms_10.json"
with open(os.path.join(DATA_PATH, transform_file_path)) as f:
    transform = json.load(f)

with open(GROUND_TRUTH_PATH) as f:
    ground_truth = json.load(f)



Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/jackal_color_wheel_training_data_1/plane-nerf/2024-02-19_174621/nerfstudio_models/step-000009999.ckpt

In [9]:
output = []

for f in range(len(transform["frames"])):
    transform_dup = transform.copy()
    transform_dup["frames"] = [transform["frames"][f]]
        
    pipeline = load_eval_image_into_pipeline(pipeline,DATA_PATH,transform_dup)

    config.pipeline.datamanager.pixel_sampler.num_rays_per_batch = 4096 

    trainer = load_data_into_trainer(
        config,
        pipeline,
        plane_optimizer = True
    )
    trainer.pipeline.datamanager.KERNEL_SIZE = 5
    trainer.pipeline.datamanager.THRESHOLD = 40
    trainer.pipeline.datamanager.METHOD = "sift"

    clear_output(wait=True)
    print(f)

    tf = ground_truth["frames"][f]["transform_matrix"]
    tf = np.asarray(tf)
    tf = tf[:3, :4 ]
    ground_truth_poses = [tf]
    ground_truth_poses = torch.tensor(ground_truth_poses).to(pipeline.device)
    
    ans = inerf(trainer, ITERATION=1000, GROUND_TRUTH_POSE=ground_truth_poses)
    output.append(ans["store"].detach().cpu().numpy().flatten())
    np.savetxt(os.path.join(MODEL_PATH,SAVE_FILE_NAME), np.asarray(output), delimiter=",")

0
{'focal_length': 4.74, 'pixel_width': 0.0042, 'pixel_height': 0.0042, 'w': 1280, 'h': 720, 'fov_x': 1.031736666064982, 'fl_x': 1128.5714285714287, 'fl_y': 1128.5714285714287, 'cx': 640, 'cy': 360, 'camera_model': 'OPENCV', 'frames': [{'file_path': 'images/0.png', 'transform_matrix': [[0.871743250061576, 0.33310636090871326, -0.3593102535333985, -1.5590494636744023], [-0.4894834370576362, 0.6245205167954374, -0.6085886040239361, -2.3829222778838326], [0.02167189004965052, 0.7064094254918419, 0.7074715914847479, 2.5], [0.0, 0.0, 0.0, 1.0]], 'mask_path': 'masks/0.png'}]}
tensor([[[ 0.7735,  0.4365, -0.4595, -1.8563],
         [-0.6334,  0.5572, -0.5369, -2.0390],
         [ 0.0217,  0.7064,  0.7075,  2.5000]]], device='cuda:0',
       dtype=torch.float64)
Number of keypoints:  87
Number of rays:  4275
Reduce the number of rays
Final number of rays:  4096
Average translation error:  tensor(0.1000, device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)
Average rotation error:  ten

In [ ]:
print(ans["store"])

tensor([[1.0000e+00, 1.5429e-07, 6.4541e-05],
        [2.0000e+00, 4.9489e-03, 9.6782e-04],
        [3.0000e+00, 9.8735e-03, 1.9926e-03],
        ...,
        [9.9800e+02, 1.0342e-02, 4.4622e-03],
        [9.9900e+02, 1.0406e-02, 4.4883e-03],
        [1.0000e+03, 1.0353e-02, 4.4483e-03]], dtype=torch.float64)
